<a href="https://colab.research.google.com/github/kywch/geo-colab/blob/master/SafeGraph-filter-places.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
colab_requirements = [
    "apt install python3-rtree=0.8.3+ds-1", # rtree v0.8.3
    "pip install geopandas==0.7.0" # geopandas v0.7.0 
]

In [2]:
# following the post, 'Making Jupyter notebooks Google Colab ready'
# https://timsainburg.com/google%20colab.html

import os, sys, subprocess

def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())
        
if 'COLAB_GPU' in os.environ: # a quick and dirty way to check whether this is COLAB
    for i in colab_requirements:
        run_subprocess_command(i)

Reading package lists...
Building dependency tree...
Reading state information...
The following package was automatically installed and is no longer required:
libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
python3-pkg-resources
Suggested packages:
python3-setuptools
The following NEW packages will be installed:
libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
python3-pkg-resources python3-rtree
0 upgraded, 5 newly installed, 0 to remove and 35 not upgraded.
Need to get 671 kB of archives.
After this operation, 3,948 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-pkg-resources all 39.0.1-2 [9

In [3]:
import pandas as pd

import geopandas as gpd
from fiona.crs import from_epsg
from shapely.geometry import Point

from glob import glob
import time

## Load the Places dataset from Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
!ls drive/My\ Drive/SafeGraph/places

brand_info.csv						  core_poi-part4.csv.gz
CorePlacesMay2020Release-CORE_POI-2020_04-2020-05-06.zip  core_poi-part5.csv.gz
core_poi-part1.csv.gz					  README.txt
core_poi-part2.csv.gz					  SquareFeet.csv.gz
core_poi-part3.csv.gz


In [13]:
# grab the SafeGraph core places
files = glob("drive/My Drive/SafeGraph/places/core*.csv.gz")
print(files)

['drive/My Drive/SafeGraph/places/core_poi-part4.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part5.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part1.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part3.csv.gz', 'drive/My Drive/SafeGraph/places/core_poi-part2.csv.gz']


In [ ]:
# copy these files from Google Drive to colab disk
from shutil import copyfile
for f in files:
  copyfile(f, '.')

In [ ]:
place_pd = pd.read_csv(files[0])


In [14]:
place_pd = pd.read_csv(files[0])
for f in files[1:]:
    t = time.time()
    tmp_pd2 = pd.read_csv(f)
    place_pd = place_pd.append(tmp_pd2, ignore_index = True)
    print(f + ', new_len: ', str(len(place_pd)) + ', proc time: ', str(time.time() - t))

OSError: ignored

In [ ]:
place_pd.info()

### Get the census block groups of Chicago

In [ ]:
cook_cbg = gpd.read_file('https://raw.githubusercontent.com/loganpowell/census-geojson/master/GeoJSON/500k/2019/17/block-group.json', 
                   crs = from_epsg(2163)).to_crs(epsg = 3528)
# select Cook county
cook_cbg = cook_cbg[cook_cbg.COUNTYFP == '031']
cook_cbg = cook_cbg[['GEOID', 'geometry']]
cook_cbg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 3992 entries, 0 to 9688
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   GEOID     3992 non-null   object  
 1   geometry  3992 non-null   geometry
dtypes: geometry(1), object(1)
memory usage: 93.6+ KB


In [ ]:
chi_bound = gpd.read_file('https://data.cityofchicago.org/api/geospatial/ewy2-6yfk?method=export&format=GeoJSON', 
                   crs = from_epsg(2163)).to_crs(epsg = 3528)

In [ ]:
chi_cbg = gpd.sjoin(cook_cbg, chi_bound,
                    op='intersects').drop("index_right", axis = 1)
chi_cbg.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2325 entries, 0 to 9685
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   GEOID       2325 non-null   object  
 1   geometry    2325 non-null   geometry
 2   name        2325 non-null   object  
 3   objectid    2325 non-null   object  
 4   shape_area  2325 non-null   object  
 5   shape_len   2325 non-null   object  
dtypes: geometry(1), object(5)
memory usage: 127.1+ KB


### Uploading a data file from the local computer

The SafeGraph data agreement prohibits me from sharing the raw data.

In [ ]:
# upload a data file from the local computer
from google.colab import files
uploaded = files.upload()

Saving 2020-04-30-social-distancing.csv.gz to 2020-04-30-social-distancing.csv.gz


In [ ]:
!ls

2020-04-30-social-distancing.csv.gz  sample_data


### Turning device counts into the probabilities

I just use one file here, `2020-04-30-social-distancing.csv.gz`, to describe what I did.

For social distancing metrics, I focused on the `buckted_percentage_time_home` and `completely_home_device_count`.

These variables are explained in [the SafeGraph page](https://docs.safegraph.com/docs/social-distancing-metrics).
* `completely_home_device_count`: Out of the device_count, the number of devices which did not leave the geohash-7 in which their home is located during the time period.
* `buckted_percentage_time_home`: Key is range of minutes and value is device count of devices that dwelled at geohash-7 of home for the given time period, `{"<60": 0, "61-360": 0, "361-720": 10, "721-1080": 40, ">1081": 50}`. For each device, we summed the observed minutes at home across the day (whether or not these were contiguous) to get the total minutes for each device this day. Then we count how many devices are in each bucket. Beginning in v2, we include the portion of any stop within the time range regardless of whether the stop start time was in the time period.


In [ ]:
to_lit = lambda x : ast.literal_eval(x) if x is not np.nan else {}

def get_prob(input_df):

    # process the date and also get weekday
    for d in ["date_range_start", "date_range_end"]:
        input_df[d] = pd.to_datetime(input_df[d])

    input_df["date"] = pd.to_datetime(input_df["date_range_start"], utc = True)\
                   .dt.tz_convert(pytz.timezone('US/Central')).dt.date
    input_df["weekday"] = input_df["date"].apply(lambda x: x.weekday())
    
    # I focused on `buckted_percentage_time_home`, along with `completely_home_device_count`
    input_df["bucketed_percentage_time_home"] = input_df["bucketed_percentage_time_home"].apply(to_lit)
    input_df["cnt_lt_6h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["76-100"] if "76-100" in x else 0)
    input_df["cnt_6h_12h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["51-75"] if "51-75" in x else 0)
    input_df["cnt_12h_18h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["26-50"] if "26-50" in x else 0)
    input_df["cnt_ht_18h_out"] = input_df.bucketed_percentage_time_home.apply(lambda x : x["0-25"] if "0-25" in x else 0)

    # device counts were turned into these probabilities
    input_df["p_completely_home"] = input_df.completely_home_device_count / input_df.device_count
    input_df["p_lt_6h_out"] = input_df["cnt_lt_6h_out"] / (input_df["cnt_lt_6h_out"] + input_df["cnt_6h_12h_out"] + input_df["cnt_12h_18h_out"] + input_df["cnt_ht_18h_out"])
    input_df["p_6h_12h_out"] = input_df["cnt_6h_12h_out"] / (input_df["cnt_lt_6h_out"] + input_df["cnt_6h_12h_out"] + input_df["cnt_12h_18h_out"] + input_df["cnt_ht_18h_out"])
    input_df["p_ht_12h_out"] = 1 - (input_df["p_lt_6h_out"] + input_df["p_6h_12h_out"])

    input_df["p_part_time_work"] = input_df.part_time_work_behavior_devices / input_df.device_count
    input_df["p_full_time_work"] = input_df.full_time_work_behavior_devices / input_df.device_count
    input_df["p_delivery"] = input_df.delivery_behavior_devices / input_df.device_count
    
    select_columns = ['origin_census_block_group', 'date', 'device_count', 'distance_traveled_from_home', 
                      'p_completely_home', 'p_lt_6h_out', 'p_6h_12h_out', 'p_ht_12h_out', 
                      'p_part_time_work', 'p_full_time_work', 'p_delivery']

    output_df = input_df[select_columns]
    
    return output_df
    

In [ ]:
oneday_df = pd.read_csv('./2020-04-30-social-distancing.csv.gz')
oneday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219343 entries, 0 to 219342
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   origin_census_block_group                   219343 non-null  int64  
 1   date_range_start                            219343 non-null  object 
 2   date_range_end                              219343 non-null  object 
 3   device_count                                219343 non-null  int64  
 4   distance_traveled_from_home                 219324 non-null  float64
 5   bucketed_distance_traveled                  219343 non-null  object 
 6   median_dwell_at_bucketed_distance_traveled  219324 non-null  object 
 7   completely_home_device_count                219343 non-null  int64  
 8   median_home_dwell_time                      219343 non-null  int64  
 9   bucketed_home_dwell_time                    219343 non-null  object 
 

In [ ]:
# select Cook county
oneday_df = oneday_df[oneday_df.origin_census_block_group // 10000000 == 17031]
oneday_df['GEOID'] = oneday_df.origin_census_block_group.astype('str')

# filter only Chicago CBGs
oneday_df = pd.merge(chi_cbg, oneday_df,
                  left_on = 'GEOID', right_on = 'GEOID', how = 'inner')
oneday_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2315 entries, 0 to 2314
Data columns (total 26 columns):
 #   Column                                      Non-Null Count  Dtype   
---  ------                                      --------------  -----   
 0   GEOID                                       2315 non-null   object  
 1   geometry                                    2315 non-null   geometry
 2   name                                        2315 non-null   object  
 3   objectid                                    2315 non-null   object  
 4   shape_area                                  2315 non-null   object  
 5   shape_len                                   2315 non-null   object  
 6   origin_census_block_group                   2315 non-null   int64   
 7   date_range_start                            2315 non-null   object  
 8   date_range_end                              2315 non-null   object  
 9   device_count                                2315 non-null   int64 

In [ ]:
import warnings; warnings.simplefilter('ignore')

oneday_df2 = get_prob(oneday_df)
oneday_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2315 entries, 0 to 2314
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   origin_census_block_group    2315 non-null   int64  
 1   date                         2315 non-null   object 
 2   device_count                 2315 non-null   int64  
 3   distance_traveled_from_home  2314 non-null   float64
 4   p_completely_home            2315 non-null   float64
 5   p_lt_6h_out                  2315 non-null   float64
 6   p_6h_12h_out                 2315 non-null   float64
 7   p_ht_12h_out                 2315 non-null   float64
 8   p_part_time_work             2315 non-null   float64
 9   p_full_time_work             2315 non-null   float64
 10  p_delivery                   2315 non-null   float64
dtypes: float64(8), int64(2), object(1)
memory usage: 217.0+ KB


### These columns are saved into social_distancing_17031.csv from 2019-01-01 to 2020-05-26